            //** You will find below the code to generate a heatmap selecting a specific date at a specific hour **\\

Let's start by install the libraries you will need. 

In [1]:
!pip install tkcalendar
!pip install folium 
!pip install datetime

Then we will import all the libraries we will need.

In [1]:
import json 
import json
import folium
import datetime 
import webbrowser
import pandas as pd

from branca import colormap
from collections import defaultdict
from folium.plugins import HeatMap
from tkinter import *
from tkcalendar import *
from datetime import datetime

Below is a function that create a calendar to select the date the user wants the data to be displayed

In [2]:
root = Tk()  ##Create the windows that will have the calendar on it
root.title("My Calendar") ##Name of the window
root.geometry("600x400") ##Size of the calendar


cal = Calendar(root, selectmode="day", year=2020, month=12, day=22) ##Default window parameters when launching the code
cal.pack(pady=20, fill="both", expand=True) ##Parameters of the calendar aesthetic 

##Function that prints the date selected by the user
def grab_date():
    my_label.config(text=cal.get_date())
    print(cal.get_date())
    
##Code for the button to validate the choice of the user
my_button = Button(root, text="Select date", command=grab_date)
my_button.pack(pady=20)

my_label = Label(root, text="")
my_label.pack(pady=20)


root.mainloop()

##Store the date in a variable to convert it into the format that is used in the dataframe column "attributes.date_debut"
date = cal.get_date()

2/24/21


In [3]:
##Same principle as preivously but this time applied for the selection of the hours.
##In this case, this is a drop down list.

list_hours = list(range(0,24))

master = Tk()
master.title("Select the hour")
master.geometry("400x200")

variable = StringVar(master)
variable.set(list_hours[0]) # default value

w = OptionMenu(master, variable, *list_hours)
w.pack()

def ok():
    print ("You choose the hour:" + variable.get())

button = Button(master, text="OK", command=ok)
button.pack()

mainloop()

##Store the hour into a variable so the full date can be converted to the adequate format
hour = variable.get()

You choose the hour:13


Now that we have the full date selected by the user, we will convert it into the format that is used in the dataframe so we call the right date from the dataframe. 

In [4]:
##Get the full date to be transform to the right format later
full_date = date + " " +hour

date_time = datetime.strptime(full_date, "%m/%d/%y %H")

print(date_time)

2021-02-24 13:00:00


Now let's proceed to the reading of the file that contains the data provided

In [5]:
##Call the .csv file with the adequate path from your computer
df = pd.read_csv("Data_Sensors.csv", sep=",")

##remove the row when an information is missing
nan = float("NaN")
df.replace("", nan, inplace=True)
df.dropna(inplace=True)

In [6]:
##Change the format of the date within the dataframe 
##Without the import of the librarie the code won't work
import datetime

df['attributes.date_debut'] = [datetime.datetime.fromtimestamp(i /1000) for i in df['attributes.date_debut']]
df['attributes.date_fin'] = [datetime.datetime.fromtimestamp(i /1000) for i in df['attributes.date_fin']]

##Reset the index of the dataframe
df = df.reset_index() 

In [7]:
#Check in the dataframe if the values are only positive, if not, remove them.

df=df.loc[df['attributes.valeur']>=0] #Only positive values
df.reset_index(drop=True, inplace=True) #Reset index after removing the outliers


From this point, we decided to display the values only in Montpellier. 
To do so, I draw a rectangle aroung the city and recovered the coordinates from Google Maps. 
Therefore, both longitude and latitude have a minimum and maximum.

But, for the sake of visibility on the heatmap, it was decided to keep all the data, the sorting having been done on the date. Because if we kept the data only in Montpellier on a specific day, there are only 2 rows of data which is very little. 

In [8]:
##We never use the same dataframe, so we copy the original dataframe that will be modified
##We called it dfMontpellier and it contains only the data recovered near Montpellier in a define area

dfMontpellier = df
frameOfMontpellier = dfMontpellier.loc[(dfMontpellier['geometry.x'] >= 3.6) & (dfMontpellier['geometry.x'] <= 4.0) & (dfMontpellier['geometry.y'] >= 43.5) & (dfMontpellier['geometry.y'] <= 43.7)]

frameOfMontpellier.reset_index()

,level_0,index,attributes.nom_dept,attributes.code_station,attributes.nom_station,attributes.nom_com,attributes.date_debut,attributes.date_fin,attributes.nom_poll,attributes.valeur,geometry.x,geometry.y
0,36,36,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2020-11-12 18:00:00,2020-11-12 19:00:00,PM10,29.5,3.88689,43.5915
1,62,62,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2020-11-12 18:00:00,2020-11-12 19:00:00,PM2.5,10.6,3.88689,43.5915
2,83,83,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2020-11-12 19:00:00,2020-11-12 20:00:00,PM10,29.2,3.88689,43.5915
3,98,98,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2020-11-12 19:00:00,2020-11-12 20:00:00,PM2.5,15.9,3.88689,43.5915
4,116,116,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2020-11-12 20:00:00,2020-11-12 21:00:00,PM2.5,16.4,3.88689,43.5915
...,...,...,...,...,...,...,...,...,...,...,...,...
15597,349478,360781,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2021-03-30 11:00:00,2021-03-30 12:00:00,PM10,31.5,3.88689,43.5915
15598,349553,360879,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2021-03-30 11:00:00,2021-03-30 12:00:00,PM2.5,7.9,3.88689,43.5915
15599,349564,360890,HERAULT,FR50203,Montpellier - Pompignane Trafic,MONTPELLIER,2021-03-30 11:00:00,2021-03-30 12:00:00,PM2.5,21.9,3.89861,43.6100
15600,349567,360893,HERAULT,FR50203,Montpellier - Pompignane Trafic,MONTPELLIER,2021-03-30 12:00:00,2021-03-30 13:00:00,PM10,27.2,3.89861,43.6100


We will now create a dropdown list so it would be more user-friendly. Therefore, the user would be able to choose between the two types of particules: PM2.5 and PM10.

In [9]:
##Same principle as the hour drop down list above, but this time it is for particule types.

list_particules = ['PM2.5', 'PM10']

master = Tk()

variable = StringVar(master)
variable.set(list_hours[0]) # default value

w = OptionMenu(master, variable, *list_particules)
w.pack()

def ok():
    print ("The particule is: " + variable.get())

button = Button(master, text="OK", command=ok)
button.pack()

mainloop()


##Store the variable so it can be called when drawing the heatmap
particule = variable.get()

The particule is: PM10


Based on the ATMO site, it is possible to assign an index between 1 and 6 to the mean concentration retrieved. The fonction below determined this index based on the concentration and the particule type. It returns the colour associated to the index and the value. It will be used when displaying a marker to have the correspondant colour with the value, just like on the ATMO site. 

In [10]:
##Calculation of the index to know the quality of the air and then print it with markers on the Folium map below

##Gives the mean for each type for particule (PM10 and PM2.5) per day
##But for the dataset given there are not enough value only for Montpellier, so we will work with the whole value 
##On the selected date. 

dfMeanForTheDay = dfMontpellier.loc[(dfMontpellier['attributes.date_debut'] == date_time) & (dfMontpellier['attributes.nom_poll'] == particule)]

##Let's define a function that return the color of the marker according to the ATMO Air Quality calculation
##Values are only relevant for daily mean according to ATMO

def quality(value, PMType):
    if PMType == "PM2.5":
        if value <10:
            return 'lightblue', '1'
        elif 11<value<20:
            return  'green', '2'
        elif 21<value<25:
            return 'beige', '3'
        elif 26<value<50:
            return 'lightred', '4'
        elif 51<value<75:
            return 'darkred', '5'
        elif value>75:
            return 'darkpurple', '6'
    elif PMType =="PM10":
        if value <20:
            return 'lightblue', '1'
        elif 21<value<40:
            return 'green', '2'
        elif 41<value<50:
            return 'beige', '3'
        elif 51<value<100:
            return 'lightred', '4'
        elif 101<value<150:
            return 'darkred', '5'
        elif value>150:
            return 'darkpurple', '6'
    else: 
        return "The particule you entered is not known in our system"
        
colour = quality(5, 'PM2.5')
#len(dfParticuleTypePerDay.columns)
#dfMeanForTheDay
colour

('lightblue', '1')

For the city of Montpellier, we decided to display a marker with the colour code of ATMO and the value to indicate the level of pollutant.

In [11]:
##Get the mean value of the selected particule in Montpellier on the selected date 
MeanPollutant = dfMeanForTheDay["attributes.valeur"].mean()

##Computation to get the colour and the index value
MeanPollutantAtmo = quality(MeanPollutant, particule)

MeanPollutantAtmo

('lightred', '4')

Now we are preparing the display of the heatmap. To highlight the region of Occitanie, we downloaded a .json file containing all the districts of Occitanie and we added it as a layer on the Folium map. 

In [12]:
geodata = open("arrondissements-occitanie.json.txt")
geodata = json.load(geodata)

Now let's generate the heatmap. To do so, we will give to the function 3 arguments: the latitude, the longitude and the concentration of pollutant.
Then the scale of colour is created and added to the map. The JSON file that delimits the region of Occitanie is read, and added also to the map. 

In [18]:
##Get from the dataframe only the latitude, longitude and the value of the pollutant
locations = df.iloc[df.loc[(df["attributes.date_debut"] == date_time)].index,[10,9,8]]

##The message that will be displayed is the index recovered from the "quality" function defined earlier
messageInPopup = MeanPollutantAtmo[1]

##Creation of the colour scale
colorMap = colormap.StepColormap(['#50F0E6', '#50CCAA', '#F0E641', '#FF5050', '#960032', '#872181'],
                           vmin=0., vmax=60.,
                           index=[0, 7, 14, 21, 28, 35, 60], caption='step')

steps=6

gradient_map=defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = colorMap.rgb_hex_str(1/steps*i)

    
##Create the map with the location of Montpellier as the center, no changing zoom allowed, only dragging
m=folium.Map(location = [43.610769, 3.876716],tiles = 'cartodbpositron', zoom_start=8,zoom_control=False,scrollWheelZoom=False,dragging=True)

##Add the marker with the corresponding colour and the index as a pop-up message
folium.Marker([43.610769, 3.876716],popup=messageInPopup,icon=folium.Icon(color=MeanPollutantAtmo[0])).add_to(m)

##Reading the JSON file to delimitate the region of Occitanie
folium.GeoJson(geodata, style_function=lambda feature: {'fillColor': 'green','color': 'darkred','weight': 0.7}, highlight_function=lambda features: {'fillColor': 'blue','color': 'lightblue'}).add_to(m)

##Add the layer of the heatmap
HeatMap(locations, gradient=gradient_map).add_to(m)

##Add the created colour scale to the map
colorMap.add_to(m)

##Display the map to check everything is all right 
m

In [49]:
##Save the map as html 
outfp = "Map_Occitanie.html"
m.save(outfp)

##Open the in another tab to display it clearly.
url = 'Map_Occitanie.html'
webbrowser.open(url, new=2)